# Belgravia Membership Analysis
## Cleaning

In [508]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium

In [509]:
df = pd.read_excel(r'/Users/jackmelleuish/Documents/GitHub/Belgravia_Leisure/DATA 30 JUN 2023 - MEMBERS.xlsx')
print(df)

                Age  Age.1  Gender Class/Membership Membership Package  \
0     MemberLTS0001    6.0  Female    LEARN-TO-SWIM      LEARN-TO-SWIM   
1     MemberLTS0004    7.0  Female    LEARN-TO-SWIM      LEARN-TO-SWIM   
2     MemberLTS0002    7.0    Male    LEARN-TO-SWIM      LEARN-TO-SWIM   
3     MemberLTS0003    6.0    Male    LEARN-TO-SWIM      LEARN-TO-SWIM   
4     MemberLTS0006    0.0  Female    LEARN-TO-SWIM      LEARN-TO-SWIM   
...             ...    ...     ...              ...                ...   
4483  MemberGym2209   36.0  Female       Membership      Reformer Plus   
4484  MemberGym2212   20.0  Female       Membership      Reformer Plus   
4485  MemberGym2213   39.0  Female       Membership      Reformer Plus   
4486  MemberGym2214   39.0  Female       Membership      Reformer Plus   
4487  MemberGym2215   45.0  Female       Membership      Reformer Plus   

     Membership Type            Suburb  
0      LEARN-TO-SWIM       AARONS PASS  
1      LEARN-TO-SWIM         

In [510]:
df = df.drop(columns = ['Age'])
df = df.rename(columns={"Age.1": "Age"})
print(df)

       Age  Gender Class/Membership Membership Package Membership Type  \
0      6.0  Female    LEARN-TO-SWIM      LEARN-TO-SWIM   LEARN-TO-SWIM   
1      7.0  Female    LEARN-TO-SWIM      LEARN-TO-SWIM   LEARN-TO-SWIM   
2      7.0    Male    LEARN-TO-SWIM      LEARN-TO-SWIM   LEARN-TO-SWIM   
3      6.0    Male    LEARN-TO-SWIM      LEARN-TO-SWIM   LEARN-TO-SWIM   
4      0.0  Female    LEARN-TO-SWIM      LEARN-TO-SWIM   LEARN-TO-SWIM   
...    ...     ...              ...                ...             ...   
4483  36.0  Female       Membership      Reformer Plus           ADULT   
4484  20.0  Female       Membership      Reformer Plus           ADULT   
4485  39.0  Female       Membership      Reformer Plus           ADULT   
4486  39.0  Female       Membership      Reformer Plus           ADULT   
4487  45.0  Female       Membership      Reformer Plus           ADULT   

                Suburb  
0          AARONS PASS  
1                ASCOT  
2                ASCOT  
3          

In [511]:
df.groupby(['Class/Membership']).size()

Class/Membership
LEARN-TO-SWIM    1060
Membership       3428
dtype: int64

In [512]:
df.groupby(['Membership Package']).size()

Membership Package
Aquatic Access      1213
Coaching Zone         10
Full Access         2003
Full Access Teen      79
LEARN-TO-SWIM       1060
Reformer Plus        123
dtype: int64

In [513]:
df.groupby(['Membership Type']).size()

Membership Type
ADULT            1698
CONCESSION       1651
LEARN-TO-SWIM    1060
TEEN (13-15)       79
dtype: int64

In [514]:
df.groupby(['Gender']).size()

Gender
Female               2451
Male                 1977
Other                  36
Prefer Not To Say      24
dtype: int64

In [515]:
df.groupby(['Age']).size()

Age
0.0      22
1.0      75
2.0      74
3.0      79
4.0      97
         ..
89.0      1
90.0      2
91.0      1
122.0     1
123.0    24
Length: 94, dtype: int64

There are 25 people with ages above 100, we decided to removed these instances are outliers

In [516]:
df = df[df['Age'] <= 100] 
df.groupby(['Age']).size()


Age
0.0     22
1.0     75
2.0     74
3.0     79
4.0     97
        ..
87.0     2
88.0     3
89.0     1
90.0     2
91.0     1
Length: 92, dtype: int64

Fixed up all incorrect spellings and data inconsistiencies.

In [517]:
df['Suburb'] = df['Suburb'].str.upper()
df['Suburb'] = df['Suburb'].str.strip()
df = df.replace({'EAGLEHAWK NECK': 'EAGLEHAWK'}, regex=True)
df = df.replace({'GOLDENSQUARE': 'GOLDEN SQUARE'}, regex=True)
df = df.replace({'HUNTLEY': 'HUNTLY'}, regex=True)
df = df.replace({'MOAMA NSW': 'MOAMA'}, regex=True)
df = df.replace({'MOULAMEIN NSW': 'MOULAMEIN'}, regex=True)
df = df.replace({'STRATHFEILDSAYE': 'STRATHFIELDSAYE'}, regex=True)
suburbs = df.groupby(['Suburb']).size()
suburbs.to_csv('suburbs.csv')

In [518]:
Teen = df[df["Membership Type"] == 'TEEN (13-15)']
Teen.groupby(['Age']).size()

Age
0.0      1
12.0     3
13.0    12
14.0    19
15.0    19
16.0    18
17.0     4
18.0     2
50.0     1
dtype: int64

It appears two people are extreme outliers for the membership and thus we remove their instance.

In [519]:
df = df.loc[~((df['Membership Type'] == 'TEEN (13-15)') & (df['Age'] >= 19.0))]
df = df.loc[~((df['Membership Type'] == 'TEEN (13-15)') & (df['Age'] <= 11.0))]
Teen = df[df["Membership Type"] == 'TEEN (13-15)']
Teen.groupby(['Age']).size()

Age
12.0     3
13.0    12
14.0    19
15.0    19
16.0    18
17.0     4
18.0     2
dtype: int64

In [520]:
conditions = [
    (df['Membership Package'] == 'LEARN-TO-SWIM'),
    (df['Membership Package'] == 'Aquatic Access'),
    (df['Membership Package'] == 'Coaching Zone'),
    (df['Membership Package'] == 'Full Access'),
    (df['Membership Package'] == 'Full Access Teen'),
    (df['Membership Package'] == 'Reformer Plus')
    ]

values = ['LEARN-TO-SWIM', 'Aquatic Access', 'Health Club', 'Health Club', 'Health Club', 'Health Club']

df['Category'] = np.select(conditions, values)

## Analysis
### Whole Centre

In [521]:
df

,Age,Gender,Class/Membership,Membership Package,Membership Type,Suburb,Category
0,6.0,Female,LEARN-TO-SWIM,LEARN-TO-SWIM,LEARN-TO-SWIM,AARONS PASS,LEARN-TO-SWIM
1,7.0,Female,LEARN-TO-SWIM,LEARN-TO-SWIM,LEARN-TO-SWIM,ASCOT,LEARN-TO-SWIM
2,7.0,Male,LEARN-TO-SWIM,LEARN-TO-SWIM,LEARN-TO-SWIM,ASCOT,LEARN-TO-SWIM
3,6.0,Male,LEARN-TO-SWIM,LEARN-TO-SWIM,LEARN-TO-SWIM,ASCOT,LEARN-TO-SWIM
4,0.0,Female,LEARN-TO-SWIM,LEARN-TO-SWIM,LEARN-TO-SWIM,AXE CREEK,LEARN-TO-SWIM
...,...,...,...,...,...,...,...
4483,36.0,Female,Membership,Reformer Plus,ADULT,STRATHFIELDSAYE,Health Club
4484,20.0,Female,Membership,Reformer Plus,ADULT,WEDDERBURN,Health Club
4485,39.0,Female,Membership,Reformer Plus,ADULT,WEST BENDIGO,Health Club
4486,39.0,Female,Membership,Reformer Plus,ADULT,WHIPSTICK,Health Club


In [522]:
df['Age'].mean()

37.318161434977576

In [523]:
minus_LTS = df[df["Class/Membership"] != 'LEARN-TO-SWIM']
minus_LTS['Age'].mean()

46.883529411764705

In [524]:
df['binned']=pd.cut(x=df['Age'], bins=[-1,5,18,35,50, 60, 85, 100])
df1 = df.groupby(['binned']).size()
df2 = df.binned.value_counts(normalize=True)*100
binned_ages = pd.concat([df1, df2], axis=1, join='inner')
binned_ages.rename(columns={"binned": "Percentage"})

,0,Percentage
"(-1, 5]",458,10.269058
"(5, 18]",772,17.309417
"(18, 35]",875,19.618834
"(35, 50]",768,17.219731
"(50, 60]",655,14.686099
"(60, 85]",919,20.605381
"(85, 100]",13,0.291480


In [525]:
df.Gender.value_counts(normalize=True)*100


Female               54.596413
Male                 44.080717
Other                 0.784753
Prefer Not To Say     0.538117
Name: Gender, dtype: float64

In [526]:
df1 = df.groupby(['binned', 'Gender']).size()
df1

binned     Gender           
(-1, 5]    Female               241
           Male                 216
           Other                  0
           Prefer Not To Say      1
(5, 18]    Female               391
           Male                 379
           Other                  2
           Prefer Not To Say      0
(18, 35]   Female               431
           Male                 421
           Other                 14
           Prefer Not To Say      9
(35, 50]   Female               430
           Male                 327
           Other                  8
           Prefer Not To Say      3
(50, 60]   Female               378
           Male                 267
           Other                  5
           Prefer Not To Say      5
(60, 85]   Female               558
           Male                 349
           Other                  6
           Prefer Not To Say      6
(85, 100]  Female                 6
           Male                   7
           Other                  0

In [527]:
df.groupby(['Class/Membership']).size()

Class/Membership
LEARN-TO-SWIM    1060
Membership       3400
dtype: int64

In [528]:
df.groupby(['Membership Package']).size()

Membership Package
Aquatic Access      1206
Coaching Zone         10
Full Access         1986
Full Access Teen      77
LEARN-TO-SWIM       1060
Reformer Plus        121
dtype: int64

In [529]:
df.groupby(['Membership Type']).size()

Membership Type
ADULT            1688
CONCESSION       1635
LEARN-TO-SWIM    1060
TEEN (13-15)       77
dtype: int64

### Learn to Swim

In [530]:
Learn2swim = df[df['Category'] == 'LEARN-TO-SWIM']
Learn2swim['Age'].mean()

6.636792452830188

In [531]:
Learn2swim.groupby(['Age']).size()
Learn2swim['binned2']=pd.cut(x=df['Age'], bins=[-1,2,5,9,13,19, 100])
L2S1 = Learn2swim.groupby(['binned2']).size()
L2S2 = Learn2swim.binned2.value_counts(normalize=True)*100
binned_ages2 = pd.concat([L2S1, L2S2], axis=1, join='inner')
binned_ages2.rename(columns={"binned2": "Percentage"})


/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_9435/821850057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Learn2swim['binned2']=pd.cut(x=df['Age'], bins=[-1,2,5,9,13,19, 100])


,0,Percentage
"(-1, 2]",168,15.849057
"(2, 5]",288,27.169811
"(5, 9]",426,40.188679
"(9, 13]",159,15.000000
"(13, 19]",4,0.377358
"(19, 100]",15,1.415094


### Aquatics

In [532]:
Aquatics = df[df['Category'] == 'Aquatic Access']
Aquatics['Age'].mean()

51.88723051409619

In [533]:
Aquatics.groupby(['Age']).size()
Aquatics['binned2']=pd.cut(x=df['Age'], bins=[-1,5,18,35,50, 60, 85, 100])
L2S1 = Aquatics.groupby(['binned2']).size()
L2S2 = Aquatics.binned2.value_counts(normalize=True)*100
binned_ages3 = pd.concat([L2S1, L2S2], axis=1, join='inner')
binned_ages3.rename(columns={"binned2": "Percentage"})

/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_9435/282438631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Aquatics['binned2']=pd.cut(x=df['Age'], bins=[-1,5,18,35,50, 60, 85, 100])


,0,Percentage
"(-1, 5]",1,0.082919
"(5, 18]",59,4.892206
"(18, 35]",233,19.320066
"(35, 50]",185,15.339967
"(50, 60]",243,20.149254
"(60, 85]",473,39.220564
"(85, 100]",12,0.995025


In [534]:
Aquatics.groupby(['Gender']).size()

Gender
Female               654
Male                 516
Other                 25
Prefer Not To Say     11
dtype: int64

In [535]:
Aquatics.Gender.value_counts(normalize=True)*100

Female               54.228856
Male                 42.786070
Other                 2.072968
Prefer Not To Say     0.912106
Name: Gender, dtype: float64

In [536]:
Aquatics.groupby(['binned2', 'Gender']).size()

binned2    Gender           
(-1, 5]    Female                 0
           Male                   1
           Other                  0
           Prefer Not To Say      0
(5, 18]    Female                36
           Male                  21
           Other                  2
           Prefer Not To Say      0
(18, 35]   Female               101
           Male                 120
           Other                  8
           Prefer Not To Say      4
(35, 50]   Female               104
           Male                  73
           Other                  6
           Prefer Not To Say      2
(50, 60]   Female               132
           Male                 105
           Other                  4
           Prefer Not To Say      2
(60, 85]   Female               276
           Male                 189
           Other                  5
           Prefer Not To Say      3
(85, 100]  Female                 5
           Male                   7
           Other                  0

In [537]:
Aquatics.groupby(['Membership Type']).size()

Membership Type
ADULT         462
CONCESSION    744
dtype: int64

In [538]:
Aquatics.groupby(['Membership Type', 'Gender']).size()

Membership Type  Gender           
ADULT            Female               236
                 Male                 205
                 Other                 13
                 Prefer Not To Say      8
CONCESSION       Female               418
                 Male                 311
                 Other                 12
                 Prefer Not To Say      3
dtype: int64

In [539]:
Aquatics.groupby(['binned2', 'Membership Type']).size()

binned2    Membership Type
(-1, 5]    ADULT                1
           CONCESSION           0
(5, 18]    ADULT                2
           CONCESSION          57
(18, 35]   ADULT              132
           CONCESSION         101
(35, 50]   ADULT              112
           CONCESSION          73
(50, 60]   ADULT              168
           CONCESSION          75
(60, 85]   ADULT               47
           CONCESSION         426
(85, 100]  ADULT                0
           CONCESSION          12
dtype: int64

### Heath Club

In [540]:
Heath_Club = df[df['Category'] == 'Health Club']
Heath_Club.groupby(['Membership Package']).size()

Membership Package
Coaching Zone         10
Full Access         1986
Full Access Teen      77
Reformer Plus        121
dtype: int64

In [541]:
Heath_Club.groupby(['Membership Package', 'binned']).size()

Membership Package  binned   
Coaching Zone       (-1, 5]        0
                    (5, 18]        0
                    (18, 35]       0
                    (35, 50]       4
                    (50, 60]       5
                    (60, 85]       1
                    (85, 100]      0
Full Access         (-1, 5]        1
                    (5, 18]       50
                    (18, 35]     601
                    (35, 50]     526
                    (50, 60]     375
                    (60, 85]     432
                    (85, 100]      1
Full Access Teen    (-1, 5]        0
                    (5, 18]       77
                    (18, 35]       0
                    (35, 50]       0
                    (50, 60]       0
                    (60, 85]       0
                    (85, 100]      0
Reformer Plus       (-1, 5]        0
                    (5, 18]        0
                    (18, 35]      35
                    (35, 50]      46
                    (50, 60]      29
        

In [542]:
Heath_Club.groupby(['Membership Package', 'Gender']).size()

Membership Package  Gender           
Coaching Zone       Female                  7
                    Male                    3
Full Access         Female               1073
                    Male                  893
                    Other                   8
                    Prefer Not To Say      12
Full Access Teen    Female                 32
                    Male                   45
Reformer Plus       Female                112
                    Male                    8
                    Other                   1
dtype: int64

In [543]:
Heath_Club.groupby(['Membership Package', 'Membership Type']).size()

Membership Package  Membership Type
Coaching Zone       ADULT                10
Full Access         ADULT              1095
                    CONCESSION          891
Full Access Teen    TEEN (13-15)         77
Reformer Plus       ADULT               121
dtype: int64

In [544]:
geo = gpd.read_file("./data/SA2_2021_AUST_GDA2020.shp")
geo = geo[['SA2_NAME21', 'SA3_NAME21']]
geo.to_csv('SA.csv')


In [545]:
df = df.replace({'ASCOT': 'White Hills - Ascot'}, regex=True)
df = df.replace({'AARONS PASS': 'Mudgee Surrounds - East'}, regex=True)
df = df.replace({'AXE CREEK': 'Bendigo Surrounds - South'}, regex=True)
df = df.replace({'AXEDALE': 'Heathcote'}, regex=True)
df = df.replace({'BAGSHOT': 'White Hills - Ascot'}, regex=True)
df = df.replace({'BALLENDELLA': 'Rochester'}, regex=True)
df = df.replace({'BALWYN NORTH': 'Balwyn North'}, regex=True)
df = df.replace({'BARKERS CREEK': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'BET BET': 'Maryborough Surrounds'}, regex=True)
df = df.replace({'BENDIGO': 'Bendigo'}, regex=True)
df = df.replace({'BIG HILL': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'BONEO': 'Rosebud - McCrae'}, regex=True)
df = df.replace({'BOOMANOOMANA': 'Cobram'}, regex=True)
df = df.replace({'BOOROLITE': 'Mansfield (Vic.)'}, regex=True)
df = df.replace({'BOORT': 'Loddon'}, regex=True)
df = df.replace({'BRIDGEWATER': 'Loddon'}, regex=True)
df = df.replace({'Loddon NORTH': 'Loddon'}, regex=True)
df = df.replace({'Loddon ON LODDON': 'Loddon'}, regex=True)
df = df.replace({'CALFORNIA GULLY': 'California Gully - Eaglehawk'}, regex=True)
df = df.replace({'CALIVIL': 'Loddon'}, regex=True)
df = df.replace({'CAIRNLEA': 'Cairnlea'}, regex=True)
df = df.replace({'CAMPBELLS CREEK': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'CAMPBELLS FOREST': 'Loddon'}, regex=True)
df = df.replace({'CARISBROOK': 'Maryborough Surrounds'}, regex=True)
df = df.replace({'CASTLEMAINE': 'Castlemaine'}, regex=True)
df = df.replace({'CHARLTON': 'Loddon'}, regex=True)
df = df.replace({'CHEWTON': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'Castlemaine Surrounds BUSHLANDS': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'CLOVERDALE WA': 'Kewdale'}, regex=True)
df = df.replace({'COHUNA': 'Lockington - Gunbower'}, regex=True)
df = df.replace({'COOMOORA': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'COSTERFIELD': 'Heathcote'}, regex=True)
df = df.replace({'DINGEE': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'DUNOLLY': 'Maryborough Surrounds'}, regex=True)
df = df.replace({'EAGLEHAWK': 'California Gully - Eaglehawk'}, regex=True)
df = df.replace({'EAST BENDIGO': 'East Bendigo - Kennington'}, regex=True)
df = df.replace({'EAST KEILOR': 'Keilor East'}, regex=True)
df = df.replace({'ECHUCA': 'Echuna'}, regex=True)
df = df.replace({'Echuna SOUTH': 'Echuna'}, regex=True)
df = df.replace({'ELMORE': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'ELPHINSTONE': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'EMU CREEK': 'Bendigo Surrounds - South'}, regex=True)
df = df.replace({'EPPALOCK': 'Heathcote'}, regex=True)
df = df.replace({'EPPING': 'Epping - East'}, regex=True)
df = df.replace({'EPSOM': 'White Hills - Ascot'}, regex=True)
df = df.replace({'ESSENDON': 'Essendon - East'}, regex=True)
df = df.replace({'FARADAY': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'FERNIHURST': 'Loddon'}, regex=True)
df = df.replace({'FRANKSTON': 'Frankston'}, regex=True)
df = df.replace({'FLORA HILL': 'Flora Hill - Spring Gully'}, regex=True)
df = df.replace({'FOSTERVILLE': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'FRYERSTOWN': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'GOLDEN GULLY': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'GOLDEN SQUARE': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'GOORNONG': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'GREENSBOROUGH': 'Greensborough'}, regex=True)
df = df.replace({'GREEN GULLY': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'HARCOURT': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'Castlemaine Surrounds NORTH': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'HAWKER': 'Outback'}, regex=True)
df = df.replace({'HEATHCOTE': 'Heathcote'}, regex=True)
df = df.replace({'Heathcote JUNCTION': 'Heathcote'}, regex=True)
df = df.replace({'HUNTLY': 'White Hills - Ascot'}, regex=True)
df = df.replace({'White Hills - Ascot NORTH': 'White Hills - Ascot'}, regex=True)
df = df.replace({'INGLEWOOD': 'Loddon'}, regex=True)
df = df.replace({'IRONBARK': 'Bendigo'}, regex=True)
df = df.replace({'IRYMPLE': 'Mildura Surrounds'}, regex=True)
df = df.replace({'JACKASS FLAT': 'California Gully - Eaglehawk'}, regex=True)
df = df.replace({'JUNORTOUN': 'Strathfieldsaye'}, regex=True)
df = df.replace({'KANGAROO FLAT': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'KEILOR EAST': 'Keilor East'}, regex=True)
df = df.replace({'KENNINGTON': 'East Bendigo - Kennington'}, regex=True)
df = df.replace({'KIMBOLTON': 'Heathcote'}, regex=True)
df = df.replace({'KNOWSLEY': 'Heathcote'}, regex=True)
df = df.replace({'KNOXFIELD': 'Knoxfield - Scoresby'}, regex=True)
df = df.replace({'KYNETON': 'Kyneton'}, regex=True)
df = df.replace({'LAANECOORIE': 'Maryborough Surrounds'}, regex=True)
df = df.replace({'LAKE GARDENS': 'Ballarat'}, regex=True)
df = df.replace({'LANSELL PLAZA': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'LEICHARDT': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'LOCKWOOD': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'Kangaroo Flat - Golden Square SOUTH': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'LONG GULLY': 'California Gully - Eaglehawk'}, regex=True)
df = df.replace({'LONGLEA': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'MAIDEN GULLY': 'Maiden Gully'}, regex=True)
df = df.replace({'MALDON': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'MALMSBURY': 'Kyneton'}, regex=True)
df = df.replace({'MALVERN': 'Malvern East'}, regex=True)
df = df.replace({'MANDURANG': 'Bendigo Surrounds - South'}, regex=True)
df = df.replace({'Bendigo Surrounds - South SOUTH': 'Bendigo Surrounds - South'}, regex=True)
df = df.replace({'MARONG': 'Bendigo Surrounds - South'}, regex=True)
df = df.replace({'MARYBOROUGH': 'Maryborough (Vic.)'}, regex=True)
df = df.replace({'MCKENZIE HILL': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'METCALFE': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'MILDURA': 'Mildura Surrounds'}, regex=True)
df = df.replace({'MITCHAM': 'Mitcham (Vic.)'}, regex=True)
df = df.replace({'MOAMA': 'Echuna'}, regex=True)
df = df.replace({'MOLIAGUL': 'Maryborough Surrounds'}, regex=True)
df = df.replace({'MOORABBIN': 'Moorabbin - Heatherton'}, regex=True)
df = df.replace({'MOULAMEIN': 'Deniliquin Surrounds'}, regex=True)
df = df.replace({'MUCKLEFORD': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'MYERS FLAT': 'California Gully - Eaglehawk'}, regex=True)
df = df.replace({'MYRTLE CREEK': 'Bendigo Surrounds - South'}, regex=True)
df = df.replace({'NEILBOROUGH': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'NERRINA': 'Ballarat North - Invermay'}, regex=True)
df = df.replace({'NEWBRIDGE': 'Loddon'}, regex=True)
df = df.replace({'NEWSTEAD': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'NORTH Bendigo': 'Bendigo'}, regex=True)
df = df.replace({'PASCOE VALE': 'Pascoe Vale South'}, regex=True)
df = df.replace({'PRAIRIE': 'Longreach'}, regex=True)
df = df.replace({'PRESTON': 'Preston - East'}, regex=True)
df = df.replace({'QUARRY HILL': 'Bendigo'}, regex=True)
df = df.replace({'RAVENSWOOD': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'Kangaroo Flat - Golden Square SOUTH': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'RAYWOOD': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'ROCHESTER': 'Rochester'}, regex=True)
df = df.replace({'ROMSEY': 'Romsey'}, regex=True)
df = df.replace({'ROXBURGH PARK': 'Roxburgh Park - North'}, regex=True)
df = df.replace({'RYE': 'Rosebud - McCrae'}, regex=True)
df = df.replace({'SAILORS GULLY': 'California Gully - Eaglehawk'}, regex=True)
df = df.replace({'SALISBURY WEST': 'Loddon'}, regex=True)
df = df.replace({'SANDON': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'SEBASTIAN': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'SEDGWICK': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'SERPENTINE': 'Loddon'}, regex=True)
df = df.replace({'SHELBOURNE': 'Kangaroo Flat - Golden Square'}, regex=True)
df = df.replace({'SHEPPARTON': 'Shepparton - North'}, regex=True)
df = df.replace({'SPRING GULLY': 'Flora Hill - Spring Gully'}, regex=True)
df = df.replace({'SPRINGVALE': 'Springvale'}, regex=True)
df = df.replace({'STRATHDALE': 'East Bendigo - Kennington'}, regex=True)
df = df.replace({'STRATHFIELDSAYE': 'Strathfieldsaye'}, regex=True)
df = df.replace({'SUNBURY': 'Sunbury'}, regex=True)
df = df.replace({'SUTTON GRANGE': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'SWAN HILL': 'Swan Hill'}, regex=True)
df = df.replace({'TARADALE': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'TARNAGULLA': 'St Arnaud'}, regex=True)
df = df.replace({'TARRENGOWER': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'THORNBURY': 'Thornbury'}, regex=True)
df = df.replace({'TOOLLEEN': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'WALMER': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'WARANGA SHORES': 'Rushworth'}, regex=True)
df = df.replace({'WARNER QLD': 'Cashmere'}, regex=True)
df = df.replace({'WEDDERBURN': 'Loddon'}, regex=True)
df = df.replace({'WELSHMANS REEF': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'WEST Bendigo': 'Bendigo'}, regex=True)
df = df.replace({'WHIPSTICK': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'WHITE HILLS': 'White Hills - Ascot'}, regex=True)
df = df.replace({'WOODEND': 'Kyneton'}, regex=True)
df = df.replace({'WOODVALE': 'Bendigo Surrounds - North'}, regex=True)
df = df.replace({'WOOLLOOMOOLOO': 'Sydney (North) - Millers Point'}, regex=True)
df = df.replace({'YAPEEN': 'Castlemaine Surrounds'}, regex=True)
df = df.replace({'YARRABERB': 'Loddon'}, regex=True)

In [547]:
suburbs = df.groupby(['Suburb']).size()
suburbs

Suburb
Ballarat                             1
Ballarat North - Invermay            1
Balwyn North                         1
Bendigo                            568
Bendigo Surrounds - North           48
Bendigo Surrounds - South          167
Cairnlea                             1
California Gully - Eaglehawk       299
Cashmere                             1
Castlemaine                         71
Castlemaine Surrounds              156
Cobram                               1
Deniliquin Surrounds                 1
EAST Bendigo                        54
East Bendigo - Kennington          248
Echuna                               6
Epping - East                        1
Essendon - East                      2
FLOREY                               1
Flora Hill - Spring Gully          230
Frankston                            1
Greensborough                        1
Heathcote                           41
Kangaroo Flat - Golden Square     1788
Keilor East                          2
Kewdale           

In [ ]:
# geo = geo[['SA1_NAME21', 'geometry']]
# geo.to_csv('sa1.csv')
# geo = geo.rename(columns={'SA3_NAME21': 'location'})
# geo = geo.drop_duplicates(subset=['location'])
# geoJSON = geo.to_json()

# senti = pd.read_json("./data/modified_data2.json")
# senti = senti[['location', 'sentiment']]

# group = pd.merge(geo, senti, on='location', how='inner')
# group = group.groupby(['location']).mean()
# group = pd.merge(group, geo, on='location', how='inner')

# map = folium.Map(location=[-37.5, 144.5], tiles="Stamen Terrain", zoom_start=8)

# c = folium.Choropleth(
#     geo_data=geoJSON, # geoJSON 
#     name='Sentiment of Tweets', # name of plot
#     data=group, # data source
#     columns=['location','sentiment'], # the columns required
#     key_on='properties.location', # this is from the geoJSON's properties
#     fill_color='YlOrRd', # color scheme
#     nan_fill_color='black',
#     legend_name='Sentiment'
# )
# c.add_to(map)
# map.save('frontend/app/templates/SentiLocation.html')